# WIP!! Summarize all the Citibike trip data CSVs
Trying out
* Dask dataframes
* SQL
* creating summary df from monthly CSVs
* ???

In [ ]:
import pandas as pd
import os
import logging

In [78]:
DATA_DIR = 'data/'
CSV_DIR = DATA_DIR + 'tripdata_csv/'
NY_DIR = CSV_DIR + 'NY/'
NJ_DIR = CSV_DIR + 'NJ/'

DB_FILE = 'data/tripdata.db'

logging.basicConfig(level=logging.WARNING)

In [ ]:
JC_DATA = os.listdir(NJ_DIR) # NOTE: this includes Hoboken and Jersey City
NYC_DATA = os.listdir(NY_DIR)

logging.info(f'{len(JC_DATA)} Jersey City files and {len(NYC_DATA)} New York City files')

In [ ]:
# Data ranges for NYC and NJ
# NOTE: data schema changes beginning 2021-02
# See: https://citibikenyc.com/system-data
SCHEMA_CHANGE_DATE = '2021-02'
# nyc_start = (2013, 6)
# nyc_change = (2021, 2)
# nyc_end = (2022, 2)

# nj_start = (2015, 9)
# nj_change = nyc_change
# nj_end = nyc_end

In [ ]:
# CSV paths for NYC, JC (pre and post schema change)
nyc_old = sorted([NY_DIR + f for f in os.listdir(NY_DIR) if f < SCHEMA_CHANGE_DATE])
nyc_new = sorted([NY_DIR + f for f in os.listdir(NY_DIR) if f >= SCHEMA_CHANGE_DATE])

jc_old = sorted([NJ_DIR + f for f in os.listdir(NJ_DIR) if f < SCHEMA_CHANGE_DATE])
jc_new = sorted([NJ_DIR + f for f in os.listdir(NJ_DIR) if f >= SCHEMA_CHANGE_DATE])


In [ ]:
# TODO only works for old schema at the moment
def summarise_months(outfilename: str, months: list):
    '''
    Writes monthly summary given list of monthly trip data
    
    :param outfilename: where to write the summary csv 
    :param months: list of CSVs for the monthly trip data
    :return: None
    '''
    summaries = []

    for file in months:
        df = pd.read_csv(file)
        df.columns = [col.lower().replace(" ", "") for col in df.columns]
        # logging.debug(f'{file}: {list(df.columns)}')

        year_month = file.split('/')[-1].removesuffix(".csv") # YYYYMM

        summary = pd.Series(dtype=object)
        summary['datetime'] = year_month
        summary['counttrips'] = df.shape[0]
        summary['meanduration'] = df.tripduration.mean()
        summary['modestartstationid'] = df.startstationid.mode()
        summary['modestartstationname'] = df.startstationname.mode()
        summary['modestartstationlatitude'] = df.startstationlatitude.mode()
        summary['modestartstationlongitude'] = df.startstationlongitude.mode()
        summary['modeendstationid'] = df.endstationid.mode()
        summary['modeendstationname'] = df.endstationname.mode()
        summary['modeendstationlatitude'] = df.endstationlatitude.mode()
        summary['modeendstationlongitude'] = df.endstationlongitude.mode()

        if 'usertype' in df.columns:
            summary['usertypevalues'] = df.usertype.value_counts()
        elif 'member_casual' in df.columns:
            summary['usertypevalues'] = df.member_casual.value_counts()

        if 'gender' in df.columns:
            summary['gendervalues'] = df.gender.value_counts()

        summaries.append(summary)

    summary_df = pd.DataFrame()
    summary_df = summary_df.append(summaries) # TODO use concat instead to suppress warning
    summary_df.set_index('datetime')
    summary_df.to_csv(outfilename)

In [ ]:
# write summary data month by month for NYC and NJ
summarise_months(DATA_DIR + "summary_nyc_old_schema.csv", nyc_old)
summarise_months(DATA_DIR + "summary_jc_old_schema.csv", jc_old)


In [71]:
# read summary
nyc_old_schema_summary = pd.read_csv("data/summary_nyc_old_schema.csv", index_col=0)
nyc_old_schema_summary

,datetime,counttrips,meanduration,modestartstationid,modestartstationname,modestartstationlatitude,modestartstationlongitude,modeendstationid,modeendstationname,modeendstationlatitude,modeendstationlongitude,usertypevalues,gendervalues
0,2013-06,577703,1372.570279,"0 459\nName: startstationid, dtype: int64","0 W 20 St & 11 Ave\nName: startstationname,...","0 40.746745\nName: startstationlatitude, dt...","0 -74.007756\nName: startstationlongitude, d...","0 459.0\nName: endstationid, dtype: float64","0 W 20 St & 11 Ave\nName: endstationname, d...","0 40.746745\nName: endstationlatitude, dtyp...","0 -74.007756\nName: endstationlongitude, dty...",Subscriber 337384\nCustomer 240319\nNa...,1 263492\n0 240748\n2 73463\nName: g...
1,2013-07,843416,998.100704,"0 459\nName: startstationid, dtype: int64","0 W 20 St & 11 Ave\nName: startstationname,...","0 40.746745\nName: startstationlatitude, dt...","0 -74.007756\nName: startstationlongitude, d...","0 497\nName: endstationid, dtype: int64","0 E 17 St & Broadway\nName: endstationname,...","0 40.73705\nName: endstationlatitude, dtype...","0 -73.990093\nName: endstationlongitude, dty...",Subscriber 668501\nCustomer 174915\nNa...,1 511479\n0 174931\n2 157006\nName: g...
2,2013-08,1001958,979.580227,"0 497\nName: startstationid, dtype: int64",0 E 17 St & Broadway\nName: startstationnam...,"0 40.73705\nName: startstationlatitude, dty...","0 -73.990093\nName: startstationlongitude, d...","0 497\nName: endstationid, dtype: int64","0 E 17 St & Broadway\nName: endstationname,...","0 40.73705\nName: endstationlatitude, dtype...","0 -73.990093\nName: endstationlongitude, dty...",Subscriber 815902\nCustomer 186056\nNa...,1 616914\n2 198948\n0 186096\nName: g...
3,2013-09,1034359,905.020577,"0 459\nName: startstationid, dtype: int64","0 W 20 St & 11 Ave\nName: startstationname,...","0 40.746745\nName: startstationlatitude, dt...","0 -74.007756\nName: startstationlongitude, d...","0 497\nName: endstationid, dtype: int64","0 E 17 St & Broadway\nName: endstationname,...","0 40.73705\nName: endstationlatitude, dtype...","0 -73.990093\nName: endstationlongitude, dty...",Subscriber 889190\nCustomer 145169\nNa...,1 669732\n2 219400\n0 145227\nName: g...
4,2013-10,1037712,825.614754,"0 521\nName: startstationid, dtype: int64","0 8 Ave & W 31 St\nName: startstationname, ...","0 40.75045\nName: startstationlatitude, dty...","0 -73.994811\nName: startstationlongitude, d...","0 497\nName: endstationid, dtype: int64","0 E 17 St & Broadway\nName: endstationname,...","0 40.73705\nName: endstationlatitude, dtype...","0 -73.990093\nName: endstationlongitude, dty...",Subscriber 940266\nCustomer 97446\nNa...,1 714293\n2 225921\n0 97498\nName: g...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,2020-09,2488225,1366.716288,"0 435\nName: startstationid, dtype: int64","0 W 21 St & 6 Ave\nName: startstationname, ...","0 40.74174\nName: startstationlatitude, dty...","0 -73.994156\nName: startstationlongitude, d...","0 435\nName: endstationid, dtype: int64","0 W 21 St & 6 Ave\nName: endstationname, dt...","0 40.74174\nName: endstationlatitude, dtype...","0 -73.994156\nName: endstationlongitude, dty...",Subscriber 1817674\nCustomer 670551\n...,1 1436049\n2 746443\n0 305733\nName...
88,2020-10,2248869,1213.051973,"0 3141\nName: startstationid, dtype: int64","0 1 Ave & E 68 St\nName: startstationname, ...","0 40.765005\nName: startstationlatitude, dt...","0 -73.958185\nName: startstationlongitude, d...","0 435\nName: endstationid, dtype: int64","0 W 21 St & 6 Ave\nName: endstationname, dt...","0 40.74174\nName: endstationlatitude, dtype...","0 -73.994156\nName: endstationlongitude, dty...",Subscriber 1702115\nCustomer 546754\n...,1 1335176\n2 654838\n0 258855\nName...
89,2020-11,1736704,1150.961181,"0 3141\nName: startstationid, dtype: int64","0 1 Ave & E 68 St\nName: startstationname, ...","0 40.765005\nName: startstationlatitude, dt...","0 -73.958185\nName: startstationlongitude, d...","0 3141\nName: endstationid, 

In [100]:
def clobber_year(year=2019, state='NY') -> pd.DataFrame:
    """
    Writes a csv to `data/` with given `outfilename` that is all monthly trip data for that `year`

    :param year: the year for which to concatenate data files
    :param outfilename: the file to write to. E.g., 'clobber_2019.csv'
    :param state: 'NY' or 'NJ'. default 'NY'
    :return: the merged dataframe
    """

    range_start = str(year) + '-01'
    range_end = str(year) + '-13' # Not sure why I have to select 13 here...
    files = None
    if state == 'NY':
        files = sorted([NY_DIR + f for f in os.listdir(NY_DIR) if range_start <= f <= range_end])
    elif state == 'NJ':
        files = sorted([NJ_DIR + f for f in os.listdir(NJ_DIR) if range_start <= f <= range_end])
    else:
        raise IndexError(f"No data for state: {state}")

    logging.debug(f'Will merge these files: {files}, number of files: {len(files)}')

    # Concatenate all monthly data in range
    dfs = []
    for file in files:
        df = pd.read_csv(file)
        df.columns = [col.lower().replace(" ", "") for col in df.columns]
        logging.debug(f'Appending df file: {file}...')
        dfs.append(df)

    logging.debug(f'Merging dataframes...')
    clobbered = pd.concat(dfs, axis=0, ignore_index=True)
    clobbered.drop(clobbered.columns[0], axis=1, inplace=True)

    return clobbered

In [101]:
# get 2019 data for testing
year_2019 = clobber_year(2019, 'NY')
year_2019.to_csv("data/NY_2019.csv")
year_2019

DEBUG:root:Will merge these files: ['data/tripdata_csv/NY/2019-01.csv', 'data/tripdata_csv/NY/2019-02.csv', 'data/tripdata_csv/NY/2019-03.csv', 'data/tripdata_csv/NY/2019-04.csv', 'data/tripdata_csv/NY/2019-05.csv', 'data/tripdata_csv/NY/2019-06.csv', 'data/tripdata_csv/NY/2019-07.csv', 'data/tripdata_csv/NY/2019-08.csv', 'data/tripdata_csv/NY/2019-09.csv', 'data/tripdata_csv/NY/2019-10.csv', 'data/tripdata_csv/NY/2019-11.csv', 'data/tripdata_csv/NY/2019-12.csv'], number of files: 12
DEBUG:root:Appending df file: data/tripdata_csv/NY/2019-01.csv...
DEBUG:root:Appending df file: data/tripdata_csv/NY/2019-02.csv...
DEBUG:root:Appending df file: data/tripdata_csv/NY/2019-03.csv...
DEBUG:root:Appending df file: data/tripdata_csv/NY/2019-04.csv...
DEBUG:root:Appending df file: data/tripdata_csv/NY/2019-05.csv...
DEBUG:root:Appending df file: data/tripdata_csv/NY/2019-06.csv...
DEBUG:root:Appending df file: data/tripdata_csv/NY/2019-07.csv...
DEBUG:root:Appending df file: data/tripdata_csv/N

,starttime,stoptime,startstationid,startstationname,startstationlatitude,startstationlongitude,endstationid,endstationname,endstationlatitude,endstationlongitude,bikeid,usertype,birthyear,gender
0,2019-01-01 00:01:47.4010,2019-01-01 00:07:07.5810,3160.0,Central Park West & W 76 St,40.778968,-73.973747,3283.0,W 89 St & Columbus Ave,40.788221,-73.970416,15839,Subscriber,1971,1
1,2019-01-01 00:04:43.7360,2019-01-01 00:10:00.6080,519.0,Pershing Square North,40.751873,-73.977706,518.0,E 39 St & 2 Ave,40.747804,-73.973442,32723,Subscriber,1964,1
2,2019-01-01 00:06:03.9970,2019-01-01 00:15:55.4380,3171.0,Amsterdam Ave & W 82 St,40.785247,-73.976673,3154.0,E 77 St & 3 Ave,40.773142,-73.958562,27451,Subscriber,1987,1
3,2019-01-01 00:07:03.5450,2019-01-01 00:52:22.6500,504.0,1 Ave & E 16 St,40.732219,-73.981656,3709.0,W 15 St & 6 Ave,40.738046,-73.996430,21579,Subscriber,1990,1
4,2019-01-01 00:07:35.9450,2019-01-01 00:12:39.5020,229.0,Great Jones St,40.727434,-73.993790,503.0,E 20 St & Park Ave,40.738274,-73.987520,35379,Subscriber,1979,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20551692,2019-12-31 23:58:18.0160,2019-12-31 23:59:51.6360,3141.0,1 Ave & E 68 St,40.765005,-73.958185,3142.0,1 Ave & E 62 St,40.761227,-73.960940,28734,Subscriber,1985,2
20551693,2019-12-31 23:58:59.8960,2020-01-01 00:12:06.3460,490.0,8 Ave & W 33 St,40.751551,-73.993934,513.0,W 56 St & 10 Ave,40.768254,-73.988639,41418,Subscriber,1991,1
20551694,2019-12-31 23:59:03.6950,2020-01-01 00:04:54.8730,3349.0,Grand Army Plaza & Plaza St West,40.672968,-73.970880,3368.0,5 Ave & 3 St,40.672815,-73.983524,27746,Subscriber,1935,2
20551695,2019-12-31 23:59:21.3610,2020-01-01 00:25:32.9420,252.0,MacDougal St & Washington Sq,40.732264,-73.998522,366.0,Clinton Ave & Myrtle Ave,40.693261,-73.968896,28506,Subscriber,1980,1


In [ ]:
year_2019.tail()

In [ ]:
# read JC summary
jc_old_schema_summary = pd.read_csv("data/summary_nyc_old_schema.csv", index_col=0)
jc_old_schema_summary

In [ ]:
# clobber all old nyc CSVs NOTE THIS CRASHES COMPUTER


# nyc_old_dfs = []
# for file in nyc_old:
#     print(f'file {NY_DIR + file}')
#     df = pd.read_csv(NY_DIR + file)
#     nyc_old_dfs.append(df)
#
# nyc_old_df = pd.concat(nyc_old_dfs, axis=0, ignore_index=True)

In [ ]:
# import dask.dataframe as dd
# ddf = dd.read_csv(nyc_old,
#                   dtype={'birth year': 'object',
#                          'end station id': 'float64'})
#
# # columns are Sentence Cased for some CSVs and lower cased for others
# ddf = ddf.rename(columns=str.lower)

In [ ]:
# ddf.describe().compute()